In [1]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 14.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requi

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from gensim.models.ldamodel import LdaModel

# Load the LDA model
loaded_lda_model = LdaModel.load('/content/drive/MyDrive/Colab Notebooks/alex/models/lda_40_topics/lda_model_40_topics.gensim')

# Now you can access information from the loaded model.
# For example, you can print the topics:
print("Topics:")
for topic_id, topic in loaded_lda_model.print_topics():
    print(f"Topic {topic_id}: {topic}")

# You can also access the dictionary (id2word)
print("\nDictionary:")
print(loaded_lda_model.id2word)

# And the number of topics
print("\nNumber of topics:")
print(loaded_lda_model.num_topics)

Topics:
Topic 13: 0.035*"espn" + 0.034*"sports" + 0.032*"tv" + 0.027*"network" + 0.024*"watch" + 0.019*"cbs" + 0.019*"usa" + 0.018*"live" + 0.018*"games" + 0.015*"coverage"
Topic 10: 0.051*"temple" + 0.050*"wright" + 0.026*"owls" + 0.025*"joe" + 0.025*"la" + 0.025*"drexel" + 0.022*"wildcats" + 0.022*"salle" + 0.017*"hawks" + 0.016*"philly"
Topic 20: 0.031*"olympic" + 0.017*"world" + 0.017*"olympics" + 0.016*"championships" + 0.016*"time" + 0.016*"gymnastics" + 0.014*"gold" + 0.013*"women" + 0.013*"wrestling" + 0.013*"won"
Topic 11: 0.038*"baseball" + 0.019*"softball" + 0.016*"tbd" + 0.015*"innings" + 0.014*"hit" + 0.014*"college_world_series" + 0.013*"inning" + 0.012*"run" + 0.011*"pitcher" + 0.011*"regional"
Topic 8: 0.029*"season" + 0.022*"injury" + 0.014*"week" + 0.013*"injuries" + 0.013*"games" + 0.012*"play" + 0.011*"practice" + 0.010*"day" + 0.009*"return" + 0.009*"left"
Topic 15: 0.041*"season" + 0.023*"guard" + 0.019*"team" + 0.018*"player" + 0.014*"rebounds" + 0.013*"freshman"

In [5]:
from gensim import corpora

# Load the corpus from the .mm file
corpus = corpora.MmCorpus('/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/corpus.mm')

print("Corpus loaded successfully.")
# You can optionally print the corpus to inspect its structure
# print(list(corpus))

Corpus loaded successfully.


In [6]:
import json
import pandas as pd

# Assuming your JSON file contains a list of lists, where each inner list is a tokenized document
# Replace '/content/drive/MyDrive/path/to/your/texts.json' with the actual path to your JSON file
texts_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/texts.json'

with open(texts_file_path, 'r') as f:
    texts = json.load(f)

print(f"Loaded {len(texts)} documents from {texts_file_path}")

Loaded 149649 documents from /content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/texts.json


In [7]:
# import pandas as pd

# # Load the tokenized data from the Excel file
# try:
#     tokenized_data_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/alex/updated_ tokenized_newspaper_data.xlsx')
#     print("Excel file loaded successfully.")
#     # Assuming the original text is in a column named 'text' or similar.
#     # Please adjust the column name if it's different in your Excel file.
#     if 'Main_text_tokenized' in tokenized_data_df.columns:
#         texts = tokenized_data_df['Main_text_tokenized'].tolist()
#         print(f"Extracted {len(texts)} documents.")
#     else:
#         texts = None
#         print("Could not find a 'text' column in the Excel file. Please make sure the column containing original texts is named 'text' or update the code.")

# except FileNotFoundError:
#     print("Error: The file was not found at the specified path.")
# except Exception as e:
#     print(f"An error occurred while reading the Excel file: {e}")

In [8]:
#TFinding the dominant topic in each sentence

def format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts): # Updated to use loaded_lda_model and corpus, and texts=None
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([sent_topics_df, pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]).to_frame().T], ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

     # Add original text to the end of the output
    if texts is not None: # Added check if texts is not None
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


# Assuming texts is available as data_words or you provide another list/Series of texts
# df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts) # Updated to use loaded_lda_model and corpus, commented out as data_words is undefined
# To run this cell, define `data_words` with your text data before calling the function.
# For example:
# data_words = ["your document text 1", "your document text 2", ...]
# Then uncomment the line below:
df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts)

# Format
if 'df_topic_sents_keywords' in locals(): # Added check if df_topic_sents_keywords is defined
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    # Show
    display(df_dominant_topic.head(20)) # Use display for better formatting

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4,0.4913,"family, school, time, life, people, told, day,...","[coaching, living, bonnie, henrickson, phone, ..."
1,1,4,0.2953,"family, school, time, life, people, told, day,...","[sidebar, ruston, la, dining, louisiana, cajun..."
2,2,16,0.265,"nba, basketball, celtics, lakers, bulls, star,...","[garden, missed, lonely, nights, season, debut..."
3,3,21,0.2142,"team, players, play, lot, time, playing, coach...","[ten, ago, chuck, daly, reclining, alongside, ..."
4,4,5,0.2549,"league, draft, nfl, season, round, million, pl...","[robinson, talk, rangers, stanley, cup, finals..."
5,5,11,0.2673,"baseball, softball, tbd, innings, hit, college...","[conference, pacific, coach_mike, gillespie, s..."
6,6,5,0.3199,"league, draft, nfl, season, round, million, pl...","[rangers, signed, center, michael, nylander, m..."
7,7,5,0.1783,"league, draft, nfl, season, round, million, pl...","[gavitt, elected, hall, modest, player, dartmo..."
8,8,22,0.1972,"women, south, basketball, wnba, connecticut, g...","[mythical, element, senior_guard, seimone, aug..."
9,9,4,0.3097,"family, school, time, life, people, told, day,...","[ncaa_women, jim, izard, diana, vines, remembe..."


In [9]:
# Display the first few rows of the dataframe to see the structure
display(df_dominant_topic.head())

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4,0.4913,"family, school, time, life, people, told, day,...","[coaching, living, bonnie, henrickson, phone, ..."
1,1,4,0.2953,"family, school, time, life, people, told, day,...","[sidebar, ruston, la, dining, louisiana, cajun..."
2,2,16,0.265,"nba, basketball, celtics, lakers, bulls, star,...","[garden, missed, lonely, nights, season, debut..."
3,3,21,0.2142,"team, players, play, lot, time, playing, coach...","[ten, ago, chuck, daly, reclining, alongside, ..."
4,4,5,0.2549,"league, draft, nfl, season, round, million, pl...","[robinson, talk, rangers, stanley, cup, finals..."


In [10]:
# Sort documents within each topic by percentage contribution in descending order
sorted_documents_by_topic = df_dominant_topic.sort_values(by=['Dominant_Topic', 'Topic_Perc_Contrib'], ascending=[True, False])

# Display the sorted document indexes for each topic
print("Document Indexes for Each Topic (Sorted by Percentage Contribution):")
for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    print(f"\nTopic {topic_num}:")
    # Get documents for the current topic and select only the 'Document_No' column
    topic_documents = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num]['Document_No']
    # Display the document indexes (first 20 as an example)
    print(topic_documents.tolist()[:20])

Document Indexes for Each Topic (Sorted by Percentage Contribution):

Topic 0:
[41663, 60670, 18707, 72012, 37592, 3062, 48272, 42023, 21372, 75518, 41524, 36857, 5007, 79649, 65983, 3064, 50073, 6516, 80141, 33327]

Topic 1:
[60212, 22335, 81958, 3512, 3510, 40863, 105625, 9885, 30483, 131005, 58687, 117858, 3450, 19345, 40434, 112493, 31667, 74007, 32025, 37539]

Topic 2:
[124848, 123374, 110854, 123120, 141071, 120059, 113385, 14384, 143913, 115101, 133151, 123121, 123126, 20420, 115098, 141147, 17921, 26306, 24961, 141490]

Topic 3:
[77022, 83656, 94717, 70761, 11292, 131802, 8994, 19333, 73620, 50074, 44086, 100379, 94996, 8995, 53822, 48384, 43636, 66527, 131787, 43348]

Topic 4:
[87936, 76784, 105102, 130222, 126048, 85340, 140751, 19603, 143254, 1126, 133552, 126779, 37175, 49849, 114544, 86124, 102913, 79638, 1977, 102999]

Topic 5:
[28432, 120843, 104108, 117624, 147245, 66450, 5488, 35493, 39421, 39498, 29280, 5419, 30662, 57960, 20047, 13824, 87192, 146400, 120308, 68973]



In [11]:
# Get the total number of documents for each dominant topic
topic_counts = df_dominant_topic['Dominant_Topic'].value_counts().sort_index()

print("Total number of documents for each topic:")
display(topic_counts)

Total number of documents for each topic:


,count
Dominant_Topic,
0,4466
1,2383
2,4145
3,2876
4,7800
5,3603
6,142
7,5288
8,3012


In [12]:
# Get the top 100 documents for each topic
top_100_documents_per_topic = pd.DataFrame()

for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    # Get the top 100 documents for the current topic
    top_100 = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num].head(100)
    # Append to the results DataFrame
    top_100_documents_per_topic = pd.concat([top_100_documents_per_topic, top_100], ignore_index=True)

# Display the top 100 documents for each topic
print("Top 100 Documents for Each Topic (Sorted by Percentage Contribution):")

# Iterate through topics and display the top documents for each
for topic_num in top_100_documents_per_topic['Dominant_Topic'].unique():
    print(f"\n--- Topic {topic_num} ---")
    display(top_100_documents_per_topic[top_100_documents_per_topic['Dominant_Topic'] == topic_num])

Top 100 Documents for Each Topic (Sorted by Percentage Contribution):

--- Topic 0 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,41663,0,0.8463,"school, schools, college, players, athletes, u...","[graduation, success, rate, gsr, developed, nc..."
1,60670,0,0.8263,"school, schools, college, players, athletes, u...","[ncaa, academic, reforms, refined, eligibility..."
2,18707,0,0.807,"school, schools, college, players, athletes, u...","[colleges, rankings, determined, federal, grad..."
3,72012,0,0.7925,"school, schools, college, players, athletes, u...","[western, sports, post, graduation, success, r..."
4,37592,0,0.792,"school, schools, college, players, athletes, u...","[athlete, graduation, rate, fell, students, ti..."
...,...,...,...,...,...
95,90457,0,0.6061,"school, schools, college, players, athletes, u...","[newton, south, school, prides, preparing, stu..."
96,82691,0,0.605,"school, schools, college, players, athletes, u...","[academic, equality, record, inroads, annual, ..."
97,20335,0,0.6048,"school, schools, college, players, athletes, u...","[apparent, loophole, national, collegiate, ath..."
98,51113,0,0.6044,"school, schools, college, players, athletes, u...","[ncaa, adopted, semester, rule, april, school,..."



--- Topic 1 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
100,60212,1,0.8513,"golf, round, shot, championship, par, club, pg...","[age, pro, pga_tour, victories, earnings, worl..."
101,22335,1,0.8393,"golf, round, shot, championship, par, club, pg...","[annika, sorenstam, mad, finished, birdies, gr..."
102,81958,1,0.8094,"golf, round, shot, championship, par, club, pg...","[defending_champion, bryce, molder, earned, pg..."
103,3512,1,0.8063,"golf, round, shot, championship, par, club, pg...","[top, somethings, chasing, tiger, woods, david..."
104,3510,1,0.7672,"golf, round, shot, championship, par, club, pg...","[top, somethings, chasing, tiger, woods, pga_t..."
...,...,...,...,...,...
195,60217,1,0.6333,"golf, round, shot, championship, par, club, pg...","[age, pro, pga_tour, victories, earnings, worl..."
196,118562,1,0.6332,"golf, round, shot, championship, par, club, pg...","[evian, les, bains, france, gamble, question, ..."
197,66816,1,0.632,"golf, round, shot, championship, par, club, pg...","[dean, wilson, earned, alternate, spot, parker..."
198,110541,1,0.632,"golf, round, shot, championship, par, club, pg...","[augusta, ga, phil, mickelson, jordan, spieth,..."



--- Topic 2 ---


...

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
3409,33318,37,0.9426,"coach, program, coaches, season, head_coach, c...","[short, seasons, coaching, serving, coach, ath..."
3410,51773,37,0.8781,"coach, program, coaches, season, head_coach, c...","[active, college_basketball_coaches, won, mult..."
3411,104394,37,0.8059,"coach, program, coaches, season, head_coach, c...","[athletic_director, feb, sports, section, arti..."
3412,40772,37,0.7758,"coach, program, coaches, season, head_coach, c...","[basketball, fired, coach, wayne, morgan, staf..."
3413,38683,37,0.7166,"coach, program, coaches, season, head_coach, c...","[scoreboard, colleges, dave, schrage, guided, ..."
...,...,...,...,...,...
3504,143693,37,0.5628,"coach, program, coaches, season, head_coach, c...","[officially, hired, jake, diebler, time, baske..."
3505,105571,37,0.5626,"coach, program, coaches, season, head_coach, c...","[basketball_coach, salary, boost, incentives, ..."
3506,33148,37,0.5625,"coach, program, coaches, season, head_coach, c...","[college_basketball, named, monday, eastern, b..."
3507,129236,37,0.5618,"coach, program, coaches, season, head_coach, c...","[basketball, head_coach, mark, turgeon, amid, ..."



--- Topic 38 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
3509,100402,38,0.6025,"hockey, goals, nhl, goal, boston, games, ice, ...","[champion, runner, score, providence, boston, ..."
3510,70780,38,0.6018,"hockey, goals, nhl, goal, boston, games, ice, ...","[east_regional, unh, cornell, rit, denver, wes..."
3511,13282,38,0.6007,"hockey, goals, nhl, goal, boston, games, ice, ...","[bc, bu, beanpot, ncaa_tournament, mention, ho..."
3512,20404,38,0.5948,"hockey, goals, nhl, goal, boston, games, ice, ...","[boston_college, flying, night, scoring, goals..."
3513,21652,38,0.5935,"hockey, goals, nhl, goal, boston, games, ice, ...","[college_hockey, lukas, dora, scored, winner, ..."
...,...,...,...,...,...
3604,59051,38,0.4853,"hockey, goals, nhl, goal, boston, games, ice, ...","[bc, time, boston_college_hockey_team, stakes,..."
3605,91271,38,0.485,"hockey, goals, nhl, goal, boston, games, ice, ...","[loui, eriksson, age, height, weight, position..."
3606,72065,38,0.4849,"hockey, goals, nhl, goal, boston, games, ice, ...","[denver, boston_college_hockey_team, takes, ic..."
3607,80525,38,0.4841,"hockey, goals, nhl, goal, boston, games, ice, ...","[boston_college, ferris, boston_college, coach..."



--- Topic 39 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
3609,47044,39,0.3361,"indiana, davis, sun_devils, knight, independen...","[coaching, wins, coaches, victories, spent, mi..."
3610,46403,39,0.3039,"indiana, davis, sun_devils, knight, independen...","[mexico, coach, steve, alford, won, ncaa_title..."
3611,3954,39,0.271,"indiana, davis, sun_devils, knight, independen...","[tournament, bob, knight, plan, watch, espn, m..."
3612,4878,39,0.2657,"indiana, davis, sun_devils, knight, independen...","[indiana, guard, dane, fife, defiant, support,..."
3613,1629,39,0.2533,"indiana, davis, sun_devils, knight, independen...","[main, story, meet, prez, folk, hero, villain,..."
3614,2491,39,0.2532,"indiana, davis, sun_devils, knight, independen...","[question, raised, indiana, marched, final, an..."
3615,40970,39,0.2509,"indiana, davis, sun_devils, knight, independen...","[lubbock, coaches, ran, programs, boot, camps,..."
3616,30496,39,0.2493,"indiana, davis, sun_devils, knight, independen...","[speculation, started, mike, davis, job, jeopa..."
3617,18010,39,0.246,"indiana, davis, sun_devils, knight, independen...","[row, critic, corner, teachers, bullies, disti..."
3618,15259,39,0.234,"indiana, davis, sun_devils, knight, independen...","[tempe, ariz, ned, wulk, led, ncaa_basketball_..."


In [13]:
# Define the path to save the Excel file in your Google Drive
excel_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_40_topics.xlsx'

# Save the DataFrame to an Excel file
try:
    top_100_documents_per_topic.to_excel(excel_file_path, index=False)
    print(f"Successfully saved the top 100 documents per topic to: {excel_file_path}")
except Exception as e:
    print(f"An error occurred while saving the Excel file: {e}")

Successfully saved the top 100 documents per topic to: /content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_40_topics.xlsx


##### **© 2024–2025 MD Rafsun Sheikh**
##### **Licensed under the Apache License, Version 2.0.**